In [31]:
from hydra import compose, initialize
from hydra.utils import instantiate

with initialize(config_path="config"):
    cfg = compose(config_name="config", overrides=["dataset=imagenet_val"])
    # model_config = cfg.model
    # model = instantiate(model_config.instance)
    # if "state_dict_path" in model_config:
    #     import torch

    #     model.load_state_dict(torch.load(model_config.state_dict_path))
    # model = model.eval()

    # from foolbox.models.pytorch import PyTorchModel

    # model = PyTorchModel(
    #     model=model,
    #     preprocessing=model_config.preprocessing,
    #     device="cpu",
    #     bounds=(0, 1),
    # )

    dataset = instantiate(cfg.dataset)
    print(dataset)


Dataset ImageNette
    Number of datapoints: 100
    Root location: datasets/imagenet/val
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(256, 256))
           )


In [19]:
from torchvision.transforms import CenterCrop

In [ ]:
CenterCrop()